# Click-Through Rate Prediction

> In online advertising, click-through rate (CTR) is a very important metric for evaluating ad performance. As a result, click prediction systems are essential and widely used for sponsored search and real-time bidding.
>
> [Link](https://www.kaggle.com/c/avazu-ctr-prediction)

## Libraries

 * **Numpy:** Useful for algebra and other mathematical utilities
 * **Pandas:** Library that enables working with dataframes
 * **Dask:** Provides functionality that mimics numpy arrays and pandas dataframes, while performing out-of-core computations
 * **Matplotlib:** Useful for fast and non-interactive visualizations
 * **Plotly:** Visualization library, with a lot of interactive functionality
 * **Sci-kit Learn:** Library with machine learning algorithms, useful, e.g., for exploratory and predictive data analysis

Start by clearing variables from previous runs 

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [27]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import matplotlib
import matplotlib.pyplot as plt
import sklearn

Set Matplotly and Plotly to be used inline throughou the notebook:

In [25]:
# Matplotlib
%matplotlib inline

# Pyplot
init_notebook_mode(connected=True)

Beautify Matplotlib:

In [57]:
matplotlib.style.use('ggplot')

Import the dataset using Pandas dataframes, with proper configuration:

In [58]:
date_parser = lambda x: pd.datetime.strptime(x, '%y%m%d%H')

Datatypes for the data features. These were obtained through observation of the data contents, and in the case of integers, of the integers feature ranges.

In [59]:
data_types = {
    'id': np.str,
    'click': np.bool_,
    'hour': np.str,
    'C1': np.uint16,
    'banner_pos': np.uint16,
    'site_id': np.object,
    'site_domain': np.object,
    'site_category': np.object,
    'app_id': np.object,
    'app_domain': np.object,
    'app_category': np.object,
    'device_id': np.object,
    'device_model': np.object,
    'device_type': np.uint16,
    'device_conn_type': np.uint16,
    'C14': np.uint16,
    'C15': np.uint16,
    'C16': np.uint16,
    'C17': np.uint16,
    'C18': np.uint16,
    'C19': np.uint16,
    'C20': np.uint16,
    'C21': np.uint16    
}

In [7]:
%%time
train_df = pd.read_csv('./data/train/train.csv',
                       dtype=data_types,
                       parse_dates=['hour'],
                       date_parser=date_parser)

<string>:2: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



Wall time: 12min 53s


Extract some basic information about the data

In [54]:
train_df.iloc[:, 10:20].head()

,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17
0,07d7df22,a99f214a,ddd2926e,44956a24,1,2,15706,320,50,1722
1,07d7df22,a99f214a,96809ac8,711ee120,1,0,15704,320,50,1722
2,07d7df22,a99f214a,b3cf8def,8a4875bd,1,0,15704,320,50,1722
3,07d7df22,a99f214a,e8275b8f,6332421a,1,0,15706,320,50,1722
4,07d7df22,a99f214a,9644d0bf,779d90c2,1,0,18993,320,50,2161


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40428967 entries, 0 to 40428966
Data columns (total 24 columns):
id                  object
click               int64
hour                datetime64[ns]
C1                  int64
banner_pos          int64
site_id             object
site_domain         object
site_category       object
app_id              object
app_domain          object
app_category        object
device_id           object
device_ip           object
device_model        object
device_type         int64
device_conn_type    int64
C14                 int64
C15                 int64
C16                 int64
C17                 int64
C18                 int64
C19                 int64
C20                 int64
C21                 int64
dtypes: datetime64[ns](1), int64(13), object(10)
memory usage: -827572848.0+ bytes


In [ ]:
train_df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1000009418151094273,0,2014-10-21,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,10000169349117863715,0,2014-10-21,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,10000371904215119486,0,2014-10-21,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,10000640724480838376,0,2014-10-21,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,10000679056417042096,0,2014-10-21,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [56]:
train_df.describe()

,click,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07,4.042897e+07
mean,1.698056e-01,1.004968e+03,2.880146e-01,1.015305e+00,3.313150e-01,1.884181e+04,3.188831e+02,6.010201e+01,2.112601e+03,1.432499e+00,2.271444e+02,5.321685e+04,8.338229e+01
std,3.754620e-01,1.094586e+00,5.063820e-01,5.274336e-01,8.547935e-01,4.959457e+03,2.127250e+01,4.729538e+01,6.094124e+02,1.326227e+00,3.510221e+02,4.995682e+04,7.028996e+01
min,0.000000e+00,1.001000e+03,0.000000e+00,0.000000e+00,0.000000e+00,3.750000e+02,1.200000e+02,2.000000e+01,1.120000e+02,0.000000e+00,3.300000e+01,-1.000000e+00,1.000000e+00
25%,0.000000e+00,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,1.692000e+04,3.200000e+02,5.000000e+01,1.863000e+03,0.000000e+00,3.500000e+01,-1.000000e+00,2.300000e+01
50%,0.000000e+00,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,2.034600e+04,3.200000e+02,5.000000e+01,2.323000e+03,2.000000e+00,3.900000e+01,1.000480e+05,6.100000e+01
75%,0.000000e+00,1.005000e+03,1.000000e+00,1.000000e+00,0.000000e+00,2.189400e+04,3.200000e+02,5.000000e+01,2.526000e+03,3.000000e+00,1.710000e+02,1.000930e+05,1.010000e+02
max,1.000000e+00,1.012000e+03,7.000000e+00,5.000000e+00,5.000000e+00,2.405200e+04,1.024000e+03,1.024000e+03,2.758000e+03,3.000000e+00,1.959000e+03,1.002480e+05,2.550000e+02


In [ ]:
iplot([go.Histogram(x=train_df['click'])])